# Семинар 10: генераторы, итераторы, оператор yield

### Глава 0: нерасказанное про классы *

Множественное наследование (+ миксины)

In [ ]:
from dataclasses import dataclass


@dataclass
class EmailMixin:
    email: str


@dataclass
class BasePerson:
    name: str
    age: int


@dataclass
class Person(EmailMixin, BasePerson):
    def __str__(self):
        return f"{self.name}, age {self.age}, email {self.email}"

In [ ]:
email = EmailMixin("email")  # не следует создавать standalone mixin

In [ ]:
me = Person(name="Jon", age=22, email="jon@hse.ru")

print(me)

Jon, age 22, email jon@hse.ru


In [ ]:
class A:
    def __init__(self):
        print("started call init A")
        super().__init__()
        print("ended call init A")

class B:
    def __init__(self):
        print("started call init B")
        # super().__init__()
        print("ended call init B")

class C(A, B):
    def __init__(self):
        print("started call init C")
        super().__init__()
        print("ended call init C")

In [ ]:
c = C()

started call init C
started call init A
started call init B
ended call init B
ended call init A
ended call init C


In [ ]:
# class A:
#     def __init__(self):
#         print("called init A")

# class B:
#     def __init__(self):
#         print("called init B")

# class C(A, B):
#     def __init__(self):
#         super().__init__()
#         super(A, self).__init__()

In [ ]:
# c = C()

Обращение к родителю через super()

In [ ]:
from dataclasses import dataclass

@dataclass
class BasePerson:
    name: str
    age: int

    def __str__(self):
        return f"{self.name}, age {self.age}"


@dataclass
class Person(BasePerson):
    def __str__(self):
        # print("calling super")
        result = super().__str__()
        return "Person " + result

In [ ]:
me = Person(name="Tema", age=22)

print(me)

Person Tema, age 22


**Вопрос:** как будет работать если наследование множественное?

### Глава 1: генераторы

Генераторы, это "ленивые" функции, возвращающие значения on demand, когда требуется, например, распаковать их в цикле.

Пример: числа Фибоначчи:

In [ ]:
def my_range(a, b):
    print("*1*")
    while a < b:
        print("*2*")
        yield a
        print("*3*")
        a += 1

In [ ]:
rng = my_range(2, 10)

In [ ]:
next(rng)  # rng.__next__()

*1*
*2*


2

In [ ]:
print(next(rng))
print(next(rng))
print(next(rng))
print(next(rng))
print(next(rng))
print(next(rng))
print(next(rng))

*3*
*2*
4
*3*
*2*
5
*3*
*2*
6
*3*
*2*
7
*3*
*2*
8
*3*
*2*
9
*3*


StopIteration: ignored

In [ ]:
print(next(rng))

StopIteration: ignored

In [ ]:
def my_range(a, b):
    while a < b:
        yield a
        a += 1

In [ ]:
for x in my_range(2, 10):
    print(x)

2
3
4
5
6
7
8
9


In [ ]:
gen = my_range(2, 10)
while True:
    try:
        x = next(gen)
        print(x)
    except StopIteration:
        break

2
3
4
5
6
7
8
9


In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1  # <---- волшебное слово, чтобы выдать очередное число, но не выходить из функции
    yield fib_2

    for _ in range(2, max_number):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
        yield fib_2

In [ ]:
fibs = generate_fib(10)

print(next(fibs))
print(next(fibs))
print(next(fibs))
print(next(fibs))

1
1
2
3


In [ ]:
fibs = generate_fib(10)
for x in fibs:
    print(x)

1
1
2
3
5
8
13
21
34
55


In [ ]:
for x in fibs:  # второй раз уже не выведет ничего
    print(x)

In [ ]:
next(fibs)  # а next выдаст ошибку

StopIteration: ignored

In [ ]:
def generate_fib_inf():
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    while True:
        fib_1, fib_2 = fib_2, fib_1 + fib_2
        yield fib_2

In [ ]:
# for x in generate_fib_inf():
#     print(x)

А если рекурсия?

In [ ]:
def traverse_dict(d):
    if not isinstance(d, dict):
        yield d
    else:  # isinstance(d, dict) is True
        for v in d.values():
            yield from traverse_dict(v)
            # for x in traverse_dict(v):
            #     yield x

In [ ]:
d = {
    "one": {
        "two": {
            "three": "four",
            "five": "six",
        },
    },
    "seven": "eight",
}

for x in traverse_dict(d):
    print(x)

four
six
eight


In [ ]:
def flatten_sequence(*args):
    for arg in args:
        yield from arg

[x for x in flatten_sequence([1,2,3,4,5,6], (-1,-2,-3,-4,-5,-6), range(6))]

[1, 2, 3, 4, 5, 6, -1, -2, -3, -4, -5, -6, 0, 1, 2, 3, 4, 5]

In [ ]:
squares = (x ** 2 for x in range(10))

In [ ]:
for x in squares:
    print(x)

0
1
4
9
16
25
36
49
64
81


In [ ]:
# использование yield справа от знака равно
# (основа для асинхронных функций)
def create_writer():
    print("I'm gonna write something")
    while True:
        line = yield
        print(f">>> {line}")

In [ ]:
writer = create_writer()

In [ ]:
type(writer)

generator

In [ ]:
writer.send(None)

I'm gonna write something


In [ ]:
writer.send("Line 1")

>>> Line 1


In [ ]:
writer.send("Line 2")
writer.send("Line 3")
writer.send("Line 4")
writer.send("Line 5")

>>> Line 2
>>> Line 3
>>> Line 4
>>> Line 5


In [ ]:
writer.close()

In [ ]:
writer.send("Line 1")

StopIteration: ignored

### Глава 2: итераторы

In [ ]:
collection = [1, 2, 3, 4, 5]

list_iter = iter(collection)  # collection.__iter__

In [ ]:
type(list_iter)

list_iterator

In [ ]:
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))

1
2
3
4
5


In [ ]:
print(next(list_iter))

StopIteration: ignored

In [ ]:
for x in enumerate(collection):
    print(x)

(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)


In [ ]:
it = enumerate(collection)

while True:
    try:
        print(next(it))
    except StopIteration:
        break

(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)


In [ ]:
# f = open("input.txt")  # <--- и это тоже!

# f.readline()  # <--- а это фактически его next

# for x in iter(f):
#     print(x)

# f.close()

In [ ]:
class FibonacciIterator:
    def __init__(self, max_number):
        self.prev = 1
        self.cur = 1
        self.num = 0
        self.max_number = max_number

    def __next__(self):
        if self.num == self.max_number:
            raise StopIteration

        result = self.prev
        self.prev, self.cur = self.cur, self.prev + self.cur
        self.num += 1
        return result

    def __iter__(self):
        return self

In [ ]:
fib_iter = FibonacciIterator(10)

for x in fib_iter:
    print(x)

1
1
2
3
5
8
13
21
34
55


In [ ]:
class SquareIterator:
    def __init__(self, initial_number):
        # Здесь хранится промежуточное значение
        self.number_to_square = initial_number

    def __next__(self):
        # Здесь мы обновляем значение и возвращаем результат
        self.number_to_square = self.number_to_square ** 2
        return self.number_to_square

    def __iter__(self):
        return self

In [ ]:
def squares(initial_number):
    while True:
        initial_number = initial_number ** 2
        yield initial_number

In [ ]:
sq_iter = SquareIterator(2)

print(next(sq_iter))
print(next(sq_iter))
print(next(sq_iter))
print(next(sq_iter))

4
16
256
65536


Важные моменты:

1) Генераторы -- это тоже итераторы
2) Любой объект, по которому можно сделать for (list, str, dict, set и тд) -- реализует протокол итератора

In [ ]:
from collections.abc import Iterable


class MyEnumerate:
    def __init__(self, iterable: Iterable, start: int = 0):
        self.iterable = iter(iterable)
        self.start = start

    def __next__(self):
        return_value = (self.start, next(self.iterable))
        self.start += 1
        return return_value

    def __iter__(self):
        return self

In [ ]:
for x in MyEnumerate(["abc", "cde", "def"]):
    print(x)

(0, 'abc')
(1, 'cde')
(2, 'def')


### Задание 1

Написать cycle через итератор

In [ ]:
from itertools import cycle

x = cycle([1,2])
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))

1
2
1
2
1
2
1
2
1


In [ ]:
class CycleIterator(Iterable):
    def __init__(self, iterable: Iterable):
        self._iterable = iterable
        self._idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        to_return = self._iterable[self._idx]  # self._iterable.__getitem__
        self._idx = (self._idx + 1) % len(self._iterable)  # self._iterable.__len__
        return to_return

In [ ]:
it = CycleIterator([1,2])
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

1
2
1
2
1
2


### Задание 2

написать свой генератор chain, который принимает на себя через * список коллекций, а возвращает лениво их итеративную склейку, например:

```
chain([1, 2, 3], {"a", "b", "c"}) -> 1, 2, 3, a, b, c
```

In [ ]:
def my_chain(*iterables: Iterable):
    for it in iterables:
        for elem in it:
            yield elem

In [ ]:
for x in my_chain([1, 2, 3], {"a", "b", "c"}):
    print(x)

1
2
3
c
a
b


In [ ]:
isinstance([1, 2, 3], Iterable)

### Задание 3

написать свой генератор flatten, принимающий коллекцию с вложенными iterable-сущностями, а возвращающую лениво (сплющенный список), например:

```
[[1, 2, 3], [4, [5, 6]]] -> [1, 2, 3, 4, 5, 6]
```

Для удобства проверять на итерируемость можно через

```
from collections.abc import Iterable
...

if isinstance(x, Iterable):
    ...
```

In [ ]:
from collections.abc import Iterable

# a = [1, 2, 3]
# isinstance(a, Iterable)

def flatten(iterable: Iterable):
    ...